In [213]:
from IPython.display import display, Image, clear_output

In [214]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets

import warnings

import time
##Custom scripts
# from OzESI_parsing_function_old import *

# from plotting_functions import *

# dfdfplotting_functions


In [215]:
import os
import shutil

def create_subdirectory():
    """
    Filters .mzML files from the given source subdirectory (located within Projects folder)
    and moves them into appropriate subfolders ('o3on' or 'o2only') based on file names.
    """

    while True:
        # Ask user for the source subdirectory (e.g. "04_29_23/mzml")
        src_subdirectory = input("Enter the source subdirectory (e.g. '04_29_23/mzml'): ")

        # Ensure source subdirectory is within Projects folder
        src_directory = os.path.join("Projects", src_subdirectory)

        # Check if the source directory exists
        if os.path.exists(src_directory):
            user_input = input(f"The directory '{src_directory}' already exists. type y to exit | type n to try another name | (y/n): ").strip().lower()
            if user_input != 'y':
                print("Please enter a different directory.")
                continue
        else:
            print(f"The directory '{src_directory}' does not exist. I am creating it. Meow.")
            os.makedirs(src_directory)

        # Set the destination directories for o3on and o2only
        dst_directory_o3on = os.path.join(src_directory, "o3on")
        dst_directory_o2only = os.path.join(src_directory, "o2only")

        # Create the destination directories if they don't exist
        os.makedirs(dst_directory_o3on, exist_ok=True)
        os.makedirs(dst_directory_o2only, exist_ok=True)

        # Loop through the files in the source directory
        for filename in os.listdir(src_directory):
            # Check if the file ends with ".mzML"
            if filename.endswith(".mzML"):
                # Determine which folder to move the file to based on its name
                if "_o3on.mzML" in filename:
                    shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o3on, filename))
                elif "_o2only.mzML" in filename:
                    shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o2only, filename))
        break

# Example Usage:
create_subdirectory()


Please enter a different directory.
Please enter a different directory.
The directory 'Projects/n' does not exist. I am creating it. Meow.


In [222]:
import os
import pandas as pd
import numpy as np
import pymzml
from collections import defaultdict


def pre_parsing_setup():
    # Defaults
    data_base_name_location = 'lipid_database/Lipid_Database.xlsx'
    Project = './Projects/'
    Project_Name = '04_29_23'
    Project_Folder_data = Project + Project_Name + '/mzml/o3on/test_1/o3on/'
    Project_results = Project + Project_Name + '/results/test_1/'
    file_name_to_save = '04_29_23'
    tolerance = 0.3
    remove_std = True
    save_data = True
    
    # Ask if user wants to use all default settings
    use_defaults = input("Use default settings? (y/n): ").strip().lower()

    # If not using defaults, ask for custom input for each variable
    if use_defaults != 'y':
        custom_db = input(f"Use default database location '{data_base_name_location}'? (y/n): ").strip().lower()
        if custom_db == 'n':
            data_base_name_location = input("Enter custom database location: ").strip()

        custom_project = input(f"Use default project '{Project}'? (y/n): ").strip().lower()
        if custom_project == 'n':
            Project = input("Enter custom project: ").strip()

        custom_project_name = input(f"Use default project name '{Project_Name}'? (y/n): ").strip().lower()
        if custom_project_name == 'n':
            Project_Name = input("Enter custom project name: ").strip()

        Project_Folder_data = Project + Project_Name + '/mzml/o3on/'
        Project_results = Project + Project_Name + '/results/'

        custom_folder_save = input(f"Use default project results '{Project_results}'? (y/n): ").strip().lower()
        if custom_folder_save == 'n':
            Project_results = input("Enter custom folder name for project results: ").strip()

        custom_file_save = input(f"Use default file name to save results '{file_name_to_save}'? (y/n): ").strip().lower()
        if custom_file_save == 'n':
            file_name_to_save = input("Enter custom file name to save results: ").strip()

        custom_tolerance = input(f"Use default tolerance '{tolerance}'? (y/n): ").strip().lower()
        if custom_tolerance == 'n':
            tolerance = float(input("Enter custom tolerance: ").strip())

        custom_remove_std = input(f"Use default remove_std setting '{remove_std}'? (y/n): ").strip().lower()
        if custom_remove_std == 'n':
            remove_std = input("Enter custom remove_std (True/False): ").strip().lower() == 'true'

        custom_save_data = input(f"Use default save_data setting '{save_data}'? (y/n): ").strip().lower()
        if custom_save_data == 'n':
            save_data = input("Enter custom save_data (True/False): ").strip().lower() == 'true'

    # Return all configurations as a dictionary
    # Return all configurations as a dictionary
    return {
        "data_base_name_location": data_base_name_location,
        "Project": Project,
        "Project_Name": Project_Name,
        "Project_Folder_data": Project_Folder_data,
        "Project_results": Project_results,
        "file_name_to_save": file_name_to_save,
        "tolerance": tolerance,
        "remove_std": remove_std,
        "save_data": save_data
    }


In [223]:

###All functions


def read_mrm_list(filename,remove_std = True):
    mrm_list_new = pd.read_excel(filename, sheet_name=None)
    mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
    mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
    # Add underscore to middle of columns names
    mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
    # Round Parent Ion and Product Ion to 1 decimal place
    mrm_list_offical['Parent_Ion'] = np.round(mrm_list_offical['Parent_Ion'],1)
    mrm_list_offical['Product_Ion'] = np.round(mrm_list_offical['Product_Ion'],1)
    # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
    mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
    # Change column compound name to lipid
    mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
    # Make a column called Class match lipid column to lipid types
    if remove_std == True:
        lipid_class = mrm_list_offical['Class'].unique()
        lipid_class_to_keep = ['PS','PG','CE','PC', 'DAG', 'PE', 'TAG', 'FA', 'Cer', 'CAR', 'PI','SM']
        mrm_list_offical = mrm_list_offical[mrm_list_offical['Class'].isin(lipid_class_to_keep)]
    return mrm_list_offical


# Function to create an ion dictionary from an MRM database DataFrame
def create_ion_dict(mrm_database):
    ion_dict = defaultdict(list)
    # Iterate through the rows of the MRM database DataFrame
    for index, row in mrm_database.iterrows():
        # Add a tuple with Lipid and Class to the ion dictionary using Parent_Ion and Product_Ion as the key
        ion_dict[(row['Parent_Ion'], row['Product_Ion'])].append((row['Lipid'], row['Class']))
    return ion_dict

### New way to parse OzESI data
OzESI_time_df = pd.DataFrame(columns=['Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class', 'Sample_ID', 'Retention_Time', 'OzESI_Intensity'])


def mzml_parser(file_name):
    global OzESI_time_df  # Declare OzESI_time_df as a global variable
    
    rows = []
    ozesi_rows = []
    
    data_folder = os.listdir(file_name)
    data_folder.sort()
    path_to_mzml_files = file_name

    for file in data_folder:
        if file.endswith('.mzML'):
            run = pymzml.run.Reader(path_to_mzml_files + file, skip_chromatogram=False)
            q1_mz = 0
            q3_mz = 0

            for spectrum in run:
                for element in spectrum.ID.split(' '):
                    
                    if 'Q1' in element:
                        q1 = element.split('=')
                        q1_mz = np.round(float(q1[1]), 1)
                    
                    if 'Q3' in element:
                        q3 = element.split('=')
                        q3_mz = np.round(float(q3[1]), 1)
                        
                        intensity_store = np.array([intensity for _, intensity in spectrum.peaks()])
                        intensity_sum = np.sum(intensity_store)
                        
                        transition = f"{q1_mz} -> {q3_mz}"
                        sample_id = file[:-5]
                        
                        rows.append({
                            'Parent_Ion': q1_mz,
                            'Product_Ion': q3_mz,
                            'Intensity': intensity_sum,
                            'Transition': transition,
                            'Sample_ID': sample_id
                        })
                        
                        for time, intensity in spectrum.peaks():
                            ozesi_rows.append({
                                'Parent_Ion': q1_mz,
                                'Product_Ion': q3_mz,
                                'Retention_Time': time,
                                'OzESI_Intensity': intensity,
                                'Sample_ID': sample_id,
                                'Transition': transition
                            })

    df = pd.DataFrame(rows)
    OzESI_time_df = pd.DataFrame(ozesi_rows)
    print('Finished parsing mzML files\n')
    return df


# Function to check if the absolute difference between two values is within a given tolerance
def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance

# Function to match the ions in a DataFrame row with the ions in an ion dictionary
def match_ions(row, ion_dict, tolerance=0.3):
    ions = (row['Parent_Ion'], row['Product_Ion'])
    matched_lipids = []
    matched_classes = []

    # Iterate through the ion dictionary
    for key, value in ion_dict.items():
        # Check if both the Parent_Ion and Product_Ion values are within the specified tolerance
        if within_tolerance(ions[0], key[0], tolerance) and within_tolerance(ions[1], key[1], tolerance):
            # If within tolerance, extend the matched_lipids and matched_classes lists with the corresponding values
            matched_lipids.extend([match[0] for match in value])
            matched_classes.extend([match[1] for match in value])

    # If any matches were found, update the Lipid and Class columns in the row
    if matched_lipids and matched_classes:
        row['Lipid'] = ' | '.join(matched_lipids)
        row['Class'] = ' | '.join(matched_classes)

    return row

####Combined functions for Matching

def match_lipids_parser(mrm_database,df, tolerance=0.3):
    ion_dict = create_ion_dict(mrm_database)
    # Assuming you have the df DataFrame to apply the match_ions function
    df_matched = df.apply(lambda row: match_ions(row, ion_dict=ion_dict, tolerance=tolerance), axis=1)


    # df_matched = df_matched.dropna()
    
    return df_matched


def save_dataframe(df, Project_results, file_name_to_save, max_attempts=5):
    folder_path = f'data_results/data/data_matching/{Project_results}'
    os.makedirs(folder_path, exist_ok=True)

    for i in range(max_attempts):
        file_path = f'{folder_path}/{file_name_to_save}.csv'
        if not os.path.isfile(file_path):
            df.to_csv(file_path, index=False)
            print(f"Saved DataFrame to {file_path}")
            break
    else:
        print(f"Failed to save DataFrame after {max_attempts} attempts.")
        return None



def full_parse(data_base_name_location, Project_Folder_data, Project_results, file_name_to_save, tolerance, remove_std=True, save_data=False):
    mrm_database = read_mrm_list(data_base_name_location, remove_std=remove_std)
    df = mzml_parser(Project_Folder_data)
    df_matched = match_lipids_parser(mrm_database, df, tolerance=tolerance)
    
    if save_data:
        save_dataframe(df_matched, Project_results, file_name_to_save)

    return df_matched






In [225]:
# First, get configurations from pre_parsing_setup
configurations = pre_parsing_setup()

# Now, extract the values from the configurations dictionary
data_base_name_location = configurations['data_base_name_location']
Project_Folder_data = configurations['Project_Folder_data']
Project_results = configurations['Project_results']
file_name_to_save = configurations['file_name_to_save']
tolerance = configurations['tolerance']
remove_std = configurations['remove_std']
save_data = configurations['save_data']

# Now call the full_parse function with the extracted variables
df_matched = full_parse(data_base_name_location, Project_Folder_data, Project_results, file_name_to_save, tolerance, remove_std=remove_std, save_data=save_data)


/tmp/ipykernel_574/3203086109.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_574/3203086109.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_574/3203086109.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Finished parsing mzML files

Failed to save DataFrame after 5 attempts.


In [226]:

df_matched.head(None)
OzESI_time_df.head(None)



,Parent_Ion,Product_Ion,Retention_Time,OzESI_Intensity,Sample_ID,Transition
0,694.6,547.6,0.118533,254.960022,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
1,694.6,547.6,0.240367,239.120010,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
2,694.6,547.6,0.362183,260.560028,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
3,694.6,547.6,0.484017,330.260010,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
4,694.6,547.6,0.605833,353.880035,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
...,...,...,...,...,...,...
28269,902.8,603.6,19.492367,30051.921875,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28270,902.8,603.6,19.614200,30672.542969,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28271,902.8,603.6,19.736033,30213.322266,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28272,902.8,603.6,19.857850,30320.322266,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6


In [227]:
mrm_database = read_mrm_list(data_base_name_location)
tolerance = 0.3
df_oz_matched = match_lipids_parser(mrm_database, OzESI_time_df, tolerance)


/tmp/ipykernel_574/3203086109.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_574/3203086109.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_574/3203086109.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [228]:
df_oz_matched.head(None)

,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
0,NaN,NaN,254.960022,694.6,547.6,0.118533,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
1,NaN,NaN,239.120010,694.6,547.6,0.240367,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
2,NaN,NaN,260.560028,694.6,547.6,0.362183,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
3,NaN,NaN,330.260010,694.6,547.6,0.484017,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
4,NaN,NaN,353.880035,694.6,547.6,0.605833,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6
...,...,...,...,...,...,...,...,...
28269,TAG,"[TG(55:10),TG(54:3)]_FA18:1",30051.921875,902.8,603.6,19.492367,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28270,TAG,"[TG(55:10),TG(54:3)]_FA18:1",30672.542969,902.8,603.6,19.614200,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28271,TAG,"[TG(55:10),TG(54:3)]_FA18:1",30213.322266,902.8,603.6,19.736033,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6
28272,TAG,"[TG(55:10),TG(54:3)]_FA18:1",30320.322266,902.8,603.6,19.857850,FAD185-M4-WT-cortex_TG18-1_o3on,902.8 -> 603.6


In [229]:

def filter_rt(df):
    """
    Filters the DataFrame based on retention times and aggregates by max intensity for unique 'Sample_ID' and 'Transition' combinations.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with columns 'Retention_Time' and 'OzESI_Intensity'.
        
    Returns:
        pd.DataFrame: Filtered and aggregated DataFrame.
    """
    # Filter based on retention time
    filtered_df = df[(df['Retention_Time'] > 10.0) & (df['Retention_Time'] < 16.0)].copy()

    # Round the values
    filtered_df['Retention_Time'] = filtered_df['Retention_Time'].round(2)
    filtered_df['OzESI_Intensity'] = filtered_df['OzESI_Intensity'].round(0)

    # Aggregate by max intensity for unique combinations of 'Sample_ID' and 'Transition'
    filtered_df = filtered_df.groupby(['Sample_ID', 'Transition']).apply(
        lambda x: x.loc[x['OzESI_Intensity'].idxmax()]).reset_index(drop=True)

    return filtered_df


filtered_df = filter_rt(df_oz_matched)
filtered_df.head(None)


,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
0,NaN,NaN,3903.0,694.6,547.6,11.69,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
1,NaN,NaN,18586.0,696.6,549.6,12.91,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,696.6 -> 549.6
2,NaN,NaN,59211.0,698.7,551.6,14.00,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,698.7 -> 551.6
3,NaN,NaN,590.0,720.6,573.6,13.63,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,720.6 -> 573.6
4,NaN,NaN,3643.0,722.6,575.6,12.77,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,722.6 -> 575.6
...,...,...,...,...,...,...,...,...
175,TAG,"[TG(53:10),TG(52:3)]_FA18:1",38326.0,874.8,575.6,13.78,FAD185-M4-WT-cortex_TG18-1_o3on,874.8 -> 575.6
176,TAG,"[TG(53:9),TG(52:2)]_FA18:1",311677.0,876.8,577.6,14.87,FAD185-M4-WT-cortex_TG18-1_o3on,876.8 -> 577.6
177,TAG,[TG(54:5)]_FA18:1,4327.0,898.8,599.6,12.56,FAD185-M4-WT-cortex_TG18-1_o3on,898.8 -> 599.6
178,TAG,"[TG(55:11),TG(54:4)]_FA18:1",23777.0,900.8,601.6,13.65,FAD185-M4-WT-cortex_TG18-1_o3on,900.8 -> 601.6


In [230]:

import pandas as pd

def concat_dataframes(df_matched, filtered_df):
    """
    Concatenates two DataFrames along the columns.
    
    Parameters:
        df_matched (pd.DataFrame): First DataFrame.
        filtered_df (pd.DataFrame): Second DataFrame, only the 'Retention_Time' and 'OzESI_Intensity' columns will be used.
        
    Returns:
        pd.DataFrame: Concatenated DataFrame.
    """
    return pd.concat([df_matched, filtered_df[['Retention_Time', 'OzESI_Intensity']]], axis=1)

df_matched_2 = concat_dataframes(df_matched, filtered_df)
df_matched_2.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity
0,NaN,2.567996e+05,NaN,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6,11.69,3903.0
1,NaN,2.185144e+05,NaN,696.6,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,696.6 -> 549.6,12.91,18586.0
2,NaN,6.608002e+06,NaN,698.7,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,698.7 -> 551.6,14.00,59211.0
3,NaN,7.330655e+04,NaN,720.6,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,720.6 -> 573.6,13.63,590.0
4,NaN,1.428391e+05,NaN,722.6,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,722.6 -> 575.6,12.77,3643.0
...,...,...,...,...,...,...,...,...,...
175,TAG,7.176499e+05,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,FAD185-M4-WT-cortex_TG18-1_o3on,874.8 -> 575.6,13.78,38326.0
176,TAG,3.673249e+06,"[TG(53:9),TG(52:2)]_FA18:1",876.8,577.6,FAD185-M4-WT-cortex_TG18-1_o3on,876.8 -> 577.6,14.87,311677.0
177,TAG,1.398073e+05,[TG(54:5)]_FA18:1,898.8,599.6,FAD185-M4-WT-cortex_TG18-1_o3on,898.8 -> 599.6,12.56,4327.0
178,TAG,3.942574e+05,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,FAD185-M4-WT-cortex_TG18-1_o3on,900.8 -> 601.6,13.65,23777.0


In [231]:

import pandas as pd

def DB_Position_df(df_matched_2, OzESI_list=[7,9,12]):
    """
    Creates a new DataFrame to store the DB_Position and Aldehyde_Ion values,
    and calculate n-i values for the given OzESI_list.
    
    Parameters:
        df_matched_2 (pd.DataFrame): Input DataFrame.
        OzESI_list (list): List of OzESI positions.
        
    Returns:
        pd.DataFrame: Modified DataFrame with new calculated columns.
    """
    # Create a new DataFrame to store the DB_Position and Aldehyde_Ion values
    df_DB_position = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

    # Loop over the range of DB_Position values and calculate the corresponding Aldehyde_Ion values
    for i in range(3, 21):
        df_DB_position.loc[i, 'DB_Position'] = i
        df_DB_position.loc[i, 'Aldehyde_Ion'] = 26 + (14 * (i-3))

    # Loop through OzESI_list
    for i in OzESI_list:
        # Retrieve the Aldehyde_Ion value for the current DB_Position
        aldehyde_ion = df_DB_position.loc[df_DB_position["DB_Position"] == i, "Aldehyde_Ion"].values[0]

        # Calculate n-i values
        df_matched_2["n-{}".format(i)] = df_matched_2["Parent_Ion"] - aldehyde_ion

    return df_matched_2

# Example usage:
df_matched_2 = DB_Position_df(df_matched_2, OzESI_list=[7,9,12])
df_matched_2.head(None)



,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12
0,NaN,2.567996e+05,NaN,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6
1,NaN,2.185144e+05,NaN,696.6,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,696.6 -> 549.6,12.91,18586.0,614.6,586.6,544.6
2,NaN,6.608002e+06,NaN,698.7,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,698.7 -> 551.6,14.00,59211.0,616.7,588.7,546.7
3,NaN,7.330655e+04,NaN,720.6,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,720.6 -> 573.6,13.63,590.0,638.6,610.6,568.6
4,NaN,1.428391e+05,NaN,722.6,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,722.6 -> 575.6,12.77,3643.0,640.6,612.6,570.6
...,...,...,...,...,...,...,...,...,...,...,...,...
175,TAG,7.176499e+05,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,FAD185-M4-WT-cortex_TG18-1_o3on,874.8 -> 575.6,13.78,38326.0,792.8,764.8,722.8
176,TAG,3.673249e+06,"[TG(53:9),TG(52:2)]_FA18:1",876.8,577.6,FAD185-M4-WT-cortex_TG18-1_o3on,876.8 -> 577.6,14.87,311677.0,794.8,766.8,724.8
177,TAG,1.398073e+05,[TG(54:5)]_FA18:1,898.8,599.6,FAD185-M4-WT-cortex_TG18-1_o3on,898.8 -> 599.6,12.56,4327.0,816.8,788.8,746.8
178,TAG,3.942574e+05,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,FAD185-M4-WT-cortex_TG18-1_o3on,900.8 -> 601.6,13.65,23777.0,818.8,790.8,748.8


In [232]:
#Make a new column for Labels of n-#
df_matched_2['Labels'] = ''
OzESI_list = [7, 9, 12]

def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance
columns = [
    'Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class',
    'Sample_ID', 'Retention_Time', 'Intensity_OzESI', 'Mean_Retention_Time',
    'Mean_Intensity_OzESI', 'n-7', 'n-9', 'n-12', 'Labels'
]
df_OzESI_n = pd.DataFrame(columns=columns)

#Function to add lipid name
def add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3):
    df_test = df_matched_2.copy()
    df_test_2 = df_matched_2.copy()
    global df_OzESI_n
    
    for i in OzESI_list:
        df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
    for i in range(len(df_test)):
        if pd.isna(df_test.loc[i, 'Lipid']):
            parent_ion = df_test.loc[i, 'Parent_Ion']
            
            for j in range(len(df_test)):
                row_data = df_test.loc[j].copy()
                if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-7' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-9' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-12' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
    
    df_test_2.dropna(subset=['Lipid'], inplace=True)
    return df_test_2


In [233]:
#Add lipid name to df_matched 3
df_matched_3 = add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3)

/tmp/ipykernel_574/349533062.py:54: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_574/349533062.py:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_574/349533062.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [234]:
df_matched_3.head(None) 

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
27,TAG,6.869138e+05,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,
28,TAG,3.262575e+06,"[TG(51:9),TG(50:2)]_FA18:1",848.8,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,848.8 -> 549.6,13.79,409651.0,766.8,738.8,696.8,
29,TAG,8.906404e+06,"[TG(51:8),TG(50:1)]_FA18:1",850.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,850.8 -> 551.6,15.13,2236988.0,768.8,740.8,698.8,
30,TAG,7.219475e+05,[TG(52:4)]_FA18:1,872.8,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,872.8 -> 573.6,12.57,159733.0,790.8,762.8,720.8,
31,TAG,4.364672e+06,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on copy,874.8 -> 575.6,13.66,928619.0,792.8,764.8,722.8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,NaN,6.591310e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1071,NaN,6.591310e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1072,NaN,6.591310e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1073,NaN,6.591310e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7


In [235]:
df_matched_3_sorted = df_matched_3.sort_values(by=['Sample_ID','Product_Ion'])
df_matched_3_sorted.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
63,TAG,686913.811157,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,
355,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
356,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
357,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
358,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1071,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1072,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7
1073,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7


In [236]:
df_matched_4 = df_matched_3_sorted.copy()
df_matched_4['Ratios'] = None



def calculate_intensity_ratio(df):
    # Create a new column for ratios
    df['Ratios'] = pd.Series(dtype='float64')

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        lipid = row['Lipid']
        label = row['Labels']
        intensity = row['OzESI_Intensity']
        sample_id = row['Sample_ID']

        # Check if the label is n-9
        if label == 'n-9':
            # Find the corresponding row with n-7 label and same lipid name
            n7_row = df[(df['Lipid'] == lipid) & (df['Labels'] == 'n-7')& (df['Sample_ID'] == sample_id)]

            # If a matching row is found, calculate the intensity ratio
            if not n7_row.empty:
                n7_intensity = n7_row['OzESI_Intensity'].values[0]
                ratio = intensity / n7_intensity

                # Assign the ratio to the 'Ratios' column
                df.at[index, 'Ratios'] = ratio

    return df

df_matched_4 = calculate_intensity_ratio(df_matched_4)


In [237]:
df_matched_4.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
63,TAG,686913.811157,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,,NaN
355,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
356,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
357,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
358,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1071,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1072,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1073,NaN,65913.104477,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN


In [243]:
#filtering df_matched for name but not removing the n-7 ratios
def sort_by_second_tg(lipid):
    tgs = lipid.split(',')
    if len(tgs) > 1:
        return tgs[1]
    else:
        return lipid

df_matched_5 = df_matched_4.copy()
df_matched_5['Lipid'] = df_matched_5['Lipid'].apply(sort_by_second_tg)
df_matched_5.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
63,TAG,686913.811157,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,,NaN
355,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
356,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
357,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
358,NaN,256799.577103,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,NaN,65913.104477,TG(54:3)]_FA18:1,820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1071,NaN,65913.104477,TG(54:3)]_FA18:1,820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1072,NaN,65913.104477,TG(54:3)]_FA18:1,820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN
1073,NaN,65913.104477,TG(54:3)]_FA18:1,820.8,603.6,FAD185-M4-WT-cortex_TG18-1_o3on,820.8 -> 603.6,14.77,1385.0,738.8,710.8,668.8,n-7,NaN


In [244]:

# Iterate through each row in the DataFrame
for index, row in df_matched_5.iterrows():
    # Extract Lipid, Sample_ID, Labels and Ratios from the row
    lipid = row['Lipid']
    sample_id = row['Sample_ID']
    labels = row['Labels']
    ratios = row['Ratios']

    # Check if ratios is not NaN
    if not pd.isna(ratios):
        # Print out the values
        print(f'Lipid: {lipid}, Sample_ID: {sample_id}, Labels: {labels}, Ratios: {ratios}')


Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: TG(52:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 0.49462251091470555
Lipid: TG(52:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 0.49462251091470555
Lipid: TG(52:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 0.49462251091470555
Lipid: TG(52:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 0.49462251091470555
L

In [245]:
def filter_highest_ratios(df):
    # Sort the DataFrame by ratios in descending order
    df_sorted = df.sort_values(by='Ratios', ascending=False)

    # Drop duplicates keeping the first occurrence (highest ratio)
    df_filtered = df_sorted.drop_duplicates(subset=['Sample_ID', 'Lipid','Labels'], keep='first')
    df_filtered = df_filtered.sort_values(by=['Sample_ID', 'Lipid'], ascending=[True, True])

    return df_filtered

df_matched_6 = filter_highest_ratios(df_matched_5)


In [246]:
df_matched_6.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
397,NaN,9.366814e+05,TG(50:1)]_FA18:1,740.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,740.8 -> 551.6,14.95,144069.0,658.8,630.8,588.8,n-9,1.729541
65,TAG,8.906404e+06,TG(50:1)]_FA18:1,850.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,850.8 -> 551.6,15.13,2236988.0,768.8,740.8,698.8,,NaN
365,NaN,6.608002e+06,TG(50:1)]_FA18:1,698.7,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,698.7 -> 551.6,14.00,59211.0,616.7,588.7,546.7,n-12,NaN
462,NaN,7.723013e+05,TG(50:1)]_FA18:1,768.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,768.8 -> 551.6,14.92,83299.0,686.8,658.8,616.8,n-7,NaN
396,NaN,4.582103e+05,TG(50:2)]_FA18:1,738.7,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,738.7 -> 549.6,13.74,49935.0,656.7,628.7,586.7,n-9,3.244007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,NaN,4.782072e+04,[TG(52:4)]_FA18:1,790.7,573.6,FAD185-M4-WT-cortex_TG18-1_o3on,790.7 -> 573.6,14.19,388.0,708.7,680.7,638.7,n-7,NaN
1010,NaN,4.960392e+04,[TG(54:5)]_FA18:1,788.8,599.6,FAD185-M4-WT-cortex_TG18-1_o3on,788.8 -> 599.6,12.36,516.0,706.8,678.8,636.8,n-9,1.622642
177,TAG,1.398073e+05,[TG(54:5)]_FA18:1,898.8,599.6,FAD185-M4-WT-cortex_TG18-1_o3on,898.8 -> 599.6,12.56,4327.0,816.8,788.8,746.8,,NaN
945,NaN,3.809232e+04,[TG(54:5)]_FA18:1,746.8,599.6,FAD185-M4-WT-cortex_TG18-1_o3on,746.8 -> 599.6,12.03,280.0,664.8,636.8,594.8,n-12,NaN


Visualizations

In [247]:
import plotly.express as px
import os

# Create the "Plots" directory if it doesn't exist
os.makedirs("Projects/04_29_23/Plots/filtered_lipids/", exist_ok=True)


# Define color mappings for Lipid patterns
color_mapping = {
    '50': 'red',
    '51': 'brown',
    '52': 'blue',
    '53': 'purple',
    '54': 'green',
}

# Get the unique Sample_IDs
sample_ids = df_matched_6['Sample_ID'].unique()

# Loop over the unique Sample_IDs
for sample_id in sample_ids:
    
    # Filter the dataframe for the current Sample_ID
    df_sample = df_matched_6[df_matched_6['Sample_ID'] == sample_id]
    
    # Assign colors to Lipids based on patterns
    lipid_colors = []
    for lipid in df_sample['Lipid']:
        color = 'gray'  # Default color
        for pattern, pattern_color in color_mapping.items():
            if pattern in lipid:
                color = pattern_color
                break
        lipid_colors.append(color)
    
    # Create the bar plot
    fig = px.bar(df_sample, x='Lipid', y='Ratios', text='Ratios', title=f'Bar Plot for Sample_ID: {sample_id}',
                 color_discrete_map=color_mapping)
    
    # Apply colors to the bars
    fig.update_traces(
        marker_color=lipid_colors,
        texttemplate='%{text:.2f}',
        textposition='auto',
        marker_line_width=0
    )
    
    # Customize the layout
    fig.update_layout(
        uniformtext_minsize=18,
        uniformtext_mode='hide',
        xaxis=dict(
            title='Lipid',
            titlefont=dict(size=16)
        ),
        yaxis=dict(
            title='Ratios',
            titlefont=dict(size=16),
            tickfont=dict(size=16)  # Set the font size of y-axis labels
        ),
        legend=dict(
            title='Lipid Patterns',
            tracegroupgap=50,
            itemsizing='constant'
        ),
        title=dict(
            text=f'Sample_ID: {sample_id}',
            font=dict(size=20)  # Set the title font size
        )
    )
    # Save the plot as an image
    file_name = f"Projects/04_29_23/Plots/filtered_lipids/plot_{sample_id}.png"

    # Check if the file already exists
    index = 1
    while os.path.exists(file_name):
        file_name = f"Projects/04_29_23/Plots/filtered_lipids/plot_{sample_id}_{index}.png"
        index += 1

    #fig.write_image(file_name)
    # Show plot
    fig.show()


In [45]:
#########################

In [46]:
##########################
##########################
##########################
##########################
##########################
##########################

Caitlin Data

In [47]:
import pandas as pd


val_df = pd.read_excel("Caitlin.xlsx", header=1)
val_df = val_df.rename(columns={'Ratio n-9/n-7': 'Ratios'})
val_df.head(None)

,Lipid,MRM Identity,MRM Transition,RT,Relative Abundance %,n-9 isomer %,n-7 isomer %,Ratios,Sample_ID
0,TG 50:1,TG 50:1_FA 18:1,850.8→551.6,15,16.441215,67.910997,32.089003,2.116332,DOD93 5xFAD F4 Hippocampus
1,TG 50:2,TG 50:2_FA 18:1,850.8→549.6,13.8,4.155703,76.970166,23.029834,3.342194,DOD93 5xFAD F4 Hippocampus
2,TG 50:3,TG 50:3_FA 18:1,846.8→547.6,12.5,0.375821,NaN,NaN,NaN,DOD93 5xFAD F4 Hippocampus
3,TG 52:2,TG 52:2_FA 18:1,876.8→547.6,14.8,41.588725,69.849531,30.150469,2.316698,DOD93 5xFAD F4 Hippocampus
4,TG 52:3,TG 52:3_FA 18:1,874.8→575.6,13.7,7.943199,77.032611,22.967389,3.354000,DOD93 5xFAD F4 Hippocampus
...,...,...,...,...,...,...,...,...,...
112,TG 52:3,TG 52:3_FA 18:1,874.8→575.6,13.7,12.187151,87.409386,12.590614,6.942424,FAD 184 F4 WT Cerebellum
113,TG 52:4,TG 52:4_FA 18:1,872.8→573.6,nd,1.459308,NaN,NaN,NaN,FAD 184 F4 WT Cerebellum
114,TG 54:3,TG 54:3_FA 18:1,902.8→603.6,14.7,22.415748,88.152261,11.847739,7.440429,FAD 184 F4 WT Cerebellum
115,TG 54:4,TG 54:4_FA 18:1,900.8→601.6,13.5,10.833710,100.000000,0.000000,NaN,FAD 184 F4 WT Cerebellum


In [53]:
import plotly.express as px

# Assuming val_df is your DataFrame after renaming the column
fig = px.scatter(val_df, x="Lipid", y="Ratios", color="Sample_ID",
                 title="Lipid vs Ratios for each Sample_ID",
                 labels={"Lipid": "Lipid", "Ratios": "Ratios"},
                 hover_data=["Sample_ID"])

fig.show()



In [51]:
import pandas as pd

# Adding a new column 'Person' to indicate the source of data
val_df['Person'] = 'Caitlin'
df_matched_6['Person'] = 'Me'

# Selecting only the required columns
val_df = val_df[['Lipid', 'Ratios', 'Sample_ID', 'Person']]
df_matched_6 = df_matched_6[['Lipid', 'Ratios', 'Sample_ID', 'Person']]

# Concatenating the two DataFrames
validation = pd.concat([val_df, df_matched_6], ignore_index=True)

# Displaying the combined DataFrame
validation.head(None)

,Lipid,Ratios,Sample_ID,Person
0,TG 50:1,2.116332,DOD93 5xFAD F4 Hippocampus,Caitlin
1,TG 50:2,3.342194,DOD93 5xFAD F4 Hippocampus,Caitlin
2,TG 50:3,NaN,DOD93 5xFAD F4 Hippocampus,Caitlin
3,TG 52:2,2.316698,DOD93 5xFAD F4 Hippocampus,Caitlin
4,TG 52:3,3.354000,DOD93 5xFAD F4 Hippocampus,Caitlin
...,...,...,...,...
1630,[TG(52:4)]_FA18:1,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me
1631,[TG(54:5)]_FA18:1,1.631728,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me
1632,[TG(54:5)]_FA18:1,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me
1633,[TG(54:5)]_FA18:1,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me


In [57]:

import re
# Mapping for 3-letter codes
keywords_mapping = {
    'hippocampus': 'HIP',
    'hippo': 'HIP',
    'cortex': 'COR',
    'cort': 'COR',
    'cerebellum': 'CER',
    'cereb': 'CER',
    'diencephalon': 'DIE',
    'dienc': 'DIE'
}
additional_keywords = ['DOD93', 'DOD94', 'FAD185', 'FAD189']

def get_id(sample_id):
    # Iterate over the keywords_mapping dictionary
    for keyword, id_code in keywords_mapping.items():
        # Check if the keyword is present in the sample_id
        if keyword.lower() in sample_id.lower():
            return id_code
    
    # Return None if no match is found
    return None

# Function to get additional ID based on keywords
def get_additional_id(sample_id):
    for keyword in additional_keywords:
        if keyword.lower() in sample_id.lower():
            return keyword
    return None

# Create the new 'ID' column
validation['ID'] = validation['Sample_ID'].apply(get_id)

# Create the new 'ID_2' column
validation['ID_2'] = validation['Sample_ID'].apply(get_additional_id)




def reformat_lipid_name(lipid_name):
    # Find pattern like TG(50:1) within the string
    match = re.search(r'\b([A-Z]{2})\((\d+:\d+)\)', lipid_name)
    # If the pattern is found, reformat it as 'TG 50:1'
    if match:
        return f"{match.group(1)} {match.group(2)}"
    # If the pattern is not found, return the original string
    return lipid_name

# Apply the transformation to each entry in the 'Lipid' column
validation['Lipid'] = validation['Lipid'].apply(reformat_lipid_name)
def extract_f3_f4(sample_id):
    # Check if 'F3', 'F4', 'M1', 'M2', 'M3', 'M4', or 'Ms' is in the Sample_ID and return it
    if 'F3' in sample_id:
        return 'F3'
    elif 'F4' in sample_id:
        return 'F4'
    elif 'M1' in sample_id:
        return 'M1'
    elif 'M2' in sample_id:
        return 'M2'
    elif 'M3' in sample_id:
        return 'M3'
    elif 'M4' in sample_id:
        return 'M4'
    elif 'Ms' in sample_id:
        return 'Ms'
    else:
        return None


# Create a new column 'ID_3' in the DataFrame
validation['ID_3'] = validation['Sample_ID'].apply(extract_f3_f4)


# validation.to_excel("validation.xlsx")
# Display the DataFrame
validation.head(None)

,Lipid,Ratios,Sample_ID,Person,ID,ID_2,ID_3
0,TG 50:1,2.116332,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
1,TG 50:2,3.342194,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
2,TG 50:3,NaN,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
3,TG 52:2,2.316698,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
4,TG 52:3,3.354000,DOD93 5xFAD F4 Hippocampus,Caitlin,HIP,DOD93,F4
...,...,...,...,...,...,...,...
1630,TG 52:4,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me,HIP,FAD189,Ms
1631,TG 54:5,1.631728,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me,HIP,FAD189,Ms
1632,TG 54:5,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me,HIP,FAD189,Ms
1633,TG 54:5,NaN,FAD189_Ms-5xFAD-hippoTG18-1_o3on,Me,HIP,FAD189,Ms


In [58]:
import plotly.express as px
save_dir = "Projects/04_29_23/Plots/validation/"
# Fill NaN values in the 'ID', 'ID_2', and 'ID_3' columns with a default value
validation['ID'] = validation['ID'].fillna('UNKNOWN')
validation['ID_2'] = validation['ID_2'].fillna('UNKNOWN')
validation['ID_3'] = validation['ID_3'].fillna('UNKNOWN')

# Loop through unique combinations of ID, ID_2, and ID_3
unique_ids = validation[['ID', 'ID_2', 'ID_3']].drop_duplicates().values

for id1, id2, id3 in unique_ids:
    # Filter the DataFrame for the current ID, ID_2, and ID_3
    filtered_df = validation[(validation['ID'] == id1) & (validation['ID_2'] == id2) & (validation['ID_3'] == id3)]

    # Create scatter plot for the current ID, ID_2, and ID_3
    fig = px.scatter(filtered_df, x="Lipid", y="Ratios", color="Person",
                     title=f"Lipid vs Ratios for ID: {id1}, ID_2: {id2}, ID_3: {id3}",
                     labels={"Lipid": "Lipid", "Ratios": "Ratios"},
                     hover_data=["Sample_ID"])
    # Save the figure as PNG
    save_path = os.path.join(save_dir, f"plot_{id1}_{id2}_{id3}.png")
    fig.write_image(save_path, format='png')

    # Display the figure
    fig.show()
